This demo is from demo 2 in [1].

In [1]:
from SOSPy import *
from sympy import symbols, Matrix, diff
import time

## Lyapunov Function Search

Consider the system $\begin{bmatrix} \dot{x_1}\\ \dot{x_2}\\ \dot{x_3} \end{bmatrix} = \begin{bmatrix} -x_1^3-x_1x_3^2 \\ -x_2-x_1^2x_2 \\ -x_3 - \frac{3x_3}{x_3^2+1} + 3x_1^2x_3 \end{bmatrix}$ with an equilibrium at the origin. We are interesting in a quadratic Lyapunov function $V(x)$ for proving stability of the system. $V(x)$ satisfies 

\begin{align*}
    V-\epsilon(x_1^2+x_2^2+x_3^2) \geq 0 \\
    -\frac{\partial V}{\partial x_1}\dot{x_1}-\frac{\partial V}{\partial x_2}\dot{x_2}-\frac{\partial V}{\partial x_3}\dot{x_3}\geq 0
\end{align*}

In the first inequality, $\epsilon$ is any positive constant used to gaurantee positive definiteness of $V(x)$. And since $\dot{x_3}$ is a rational function, and $x_3^2+1>0$ for all $x_3$, we can reformulate the second inequality as 
$$-\frac{\partial V}{\partial x_1}(x_3^2+1)\dot{x_1}-\frac{\partial V}{\partial x_2}(x_3^2+1)\dot{x_2}-\frac{\partial V}{\partial x_3}(x_3^2+1)\dot{x_3}\geq 0$$

In this demo, we assume $V(x) = coeff_1 * x_1^2 + coeff_2 * x_2^2 + coeff_3 * x_3^2$ by defining $V$ being constructed by $[x_1^2, x_2^2, x_3^2]$. 

In [2]:
# Declare scalar variables
x1,x2,x3 = symbols("x1,x2,x3")
vartable = [x1,x2,x3]

# Use Matrix to construct the system dx/dt
# Multiply the system by (x3^2 + 1)
f = Matrix([(-x1**3-x1*x3**2)*(x3**2+1), (-x2-x1**2*x2)*(x3**2+1), (-x3+3*x1**2*x3)*(x3**2+1)-3*x3])

# =============================================
# First, initialize the sum of squares program
prog = sosprogram(vartable)

Installed SDP solvers:  ['MOSEK', 'CVXOPT', 'SCS', 'SDPA']


Then, we need to construct the Lyapunov function $V(x)$. Assume the monomials in polynomial $V(x)$ are $[x_1^2, x_2^2, x_3^2]$, use **sospolyvar** we generate $V(x) = coeff_0 * x_1^2 + coeff_1 * x_2^2 + coeff_2 * x_3^2$ (index base is 0).

In [3]:
# =============================================
# The Lyapunoc function V(x)
prog, V = sospolyvar(prog, [x1**2, x2**2, x3**2])
print(V)

coeff_0*x1**2 + coeff_1*x2**2 + coeff_2*x3**2


In [4]:
# ============================================
# Next, define SOSP constraints

# Constraint 1: V(x) - (x1^2 + x2^2 + x3^2) >= 0
prog = sosineq(prog,V-(x1**2+x2**2+x3**2))

# Constraint 2: -dV/dx*(x3^2+1)*f >= 0
V_grad = Matrix([diff(V, x1), diff(V, x2), diff(V, x3)])
expr = -(V_grad.dot(f))  # f(x) is already multiplied by (x3^2+1)
prog = sosineq(prog,expr)

# =============================================
# And Call solver
prog = sossolve(prog,verbose=0)

# =============================================
# Finally, get solution
SOLV = sosgetsol(prog,V,"V")


 Residual norm 3.3485669911504736e-15
cpusec: 0.04917
iter: 7
status: optimal
pinf: 0.0
dinf: 0.0


<IPython.core.display.Math object>

As shown, the Lyapunov function is $V(x) = 7.80619*x_1^2 + 4.76377*x_2^2 + 1.93392*x_3^2$. Different solvers may return different solutions.

### Citation:

[1]: A. Papachristodoulou, J. Anderson, G. Valmorbida, S. Prajna, P. Seiler, P. A. Parrilo, M. M. Peet, and D. Jagt, "4.2 Lyapunov Function Search," in _Sum of Squares Optimization Toolbox for MATLAB, User’s guide_, Version 4.00, 2021, pp. 34-37.